In [1]:
# !pip install torchsummary

In [2]:
##code
"""
Target : improve performance using augmentation
Result : Train acc 99.28%- Test acc- 99.33% at 15th epoch
Analysis: Augmentaion with rotation from -7 to 7 degree really helped to up the test performnce above 99.3%
"""

'\nTarget : improve performance using augmentation\nResult : \nAnalysis: \n'

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [4]:
dropout_value = 0.05
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.convblock1 = nn.Sequential(
          nn.Conv2d(in_channels=1, out_channels=4 ,kernel_size=(3,3),padding=0, bias=False), ##>>40 param, 28 input/channel size
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=1, bias=False), ##>>148 , 26
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=1, bias=False), ##>>148, 26
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.Dropout(dropout_value)
        )
        self.pool1 = nn.Sequential( ##>>0, 13
          nn.MaxPool2d(2,2),
        )
        self.convblock2 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), ##>>298 , 13
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 13
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 13
          nn.BatchNorm2d(16),
          nn.ReLU(), 
          nn.Dropout(dropout_value)       
        )
        self.pool2 = nn.Sequential( ##>>0, 6
          nn.MaxPool2d(2,2)
        )
        self.convblock3 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), ##>>298 , 6
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 6
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 6
          nn.BatchNorm2d(16),
          nn.ReLU(),
          # nn.Dropout(dropout_value)
        )
        self.pool3 = nn.Sequential( ##>>0, 3
          nn.MaxPool2d(2,2),
        )
        self.convblock4 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=8 ,kernel_size=3,padding=1, bias=False), ##>>1168 , 3
          nn.BatchNorm2d(8),
          nn.ReLU(),
          # nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=3,padding=1, bias=False), ##>>2320 , 3
          # nn.BatchNorm2d(16),
          # nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=10 ,kernel_size=3,padding=0, bias=False), ##>>1450 , 1
          # nn.ReLU()
        )

    def forward(self, x):
        x = self.convblock1(x)
        # print (x.shape)
        x = self.pool1(x)
        # print (x.shape)
        x= self.convblock2(x)
        # print (x.shape)
        x= self.pool2(x)
        # print (x.shape)
        x= self.convblock3(x)
        # print (x.shape)
        x= self.pool3(x)
        # print (x.shape)
        x= self.convblock4(x)
        # print (x.shape)
        x = x.view(-1,10)
        x = F.log_softmax(x)
        return x 


In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              36
       BatchNorm2d-2            [-1, 4, 26, 26]               8
              ReLU-3            [-1, 4, 26, 26]               0
            Conv2d-4            [-1, 8, 26, 26]             288
       BatchNorm2d-5            [-1, 8, 26, 26]              16
              ReLU-6            [-1, 8, 26, 26]               0
            Conv2d-7           [-1, 16, 26, 26]           1,152
       BatchNorm2d-8           [-1, 16, 26, 26]              32
              ReLU-9           [-1, 16, 26, 26]               0
          Dropout-10           [-1, 16, 26, 26]               0
        MaxPool2d-11           [-1, 16, 13, 13]               0
           Conv2d-12            [-1, 4, 13, 13]             576
      BatchNorm2d-13            [-1, 4, 13, 13]               8
             ReLU-14            [-1, 4,

/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_58454/3223139834.py:78: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [7]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 7, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
                        transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1)
        correct+=pred.eq(target).sum().item()
    acc = 100*correct/len(train_loader.dataset)
    print(f'Train Accuracy: {acc:.2f}%')
        
def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _,(data, target) in enumerate(test_loader):
            data, target = data.float().to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
# scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
for epoch in range(1, 15):
    print(f'Epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    # scheduler1.step()
    test(model, device, test_loader,epoch)


Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_58454/3223139834.py:78: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
loss=0.1641707718372345 batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.93it/s]  


Train Accuracy: 93.42%
Test set: Average loss: 0.0610, Accuracy: 9810/10000 (98.10%)
Epoch: 2


loss=0.01435117144137621 batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.64it/s]  


Train Accuracy: 97.93%
Test set: Average loss: 0.0413, Accuracy: 9870/10000 (98.70%)
Epoch: 3


...

Train Accuracy: 99.19%
Test set: Average loss: 0.0230, Accuracy: 9922/10000 (99.22%)
Epoch: 13


loss=0.004169031977653503 batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.62it/s]  


Train Accuracy: 99.18%
Test set: Average loss: 0.0252, Accuracy: 9926/10000 (99.26%)
Epoch: 14


loss=0.03990931808948517 batch_id=937: 100%|██████████| 938/938 [00:41<00:00, 22.62it/s]   


Train Accuracy: 99.28%
Test set: Average loss: 0.0212, Accuracy: 9933/10000 (99.33%)
